## Поиск коллокаций в тексте

Коллокации - это неслучайные сочетания слов, всречающиеся в тексте. Они чаще других являются кандидатами на неоднословные термины предметной области. 

Для начала попробуем извлечь их при помощи Tf*Idf.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pymorphy2
import re
import numpy as np
from tqdm import tqdm

In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
with open("data/lenta2018.txt", encoding="utf-8") as news_file: # Файл с новостями.
    lenta_news = [n.split("-----\n")[1] for n in news_file.read().split("=====\n")[1:]]
    
with open("data/habr_10000.txt", encoding="utf-8") as news_file: # Файл с новостями.
    habr_news = [n for n in news_file.read().split("=====\n")[1:]]
    

In [4]:
imp_POS = ['ADJF', 'ADJS', 'NOUN', 'VERB', 'PRTF', 'PRTS', 'GRND', 'PREP']


def normalizePymorphy(text):
    tokens = re.findall('[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        if str(pv[0].tag.POS) in imp_POS:
            words.append(pv[0].normal_form)
    return words    


In [5]:
lenta_news = [' '.join(normalizePymorphy(news)) for news in tqdm(lenta_news)]
habr_news = [' '.join(normalizePymorphy(news)) for news in tqdm(habr_news)]

100%|█████████████████████████████████████████| 708/708 [00:20<00:00, 33.85it/s]


In [6]:
tfv = TfidfVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                      ngram_range=(2, 3))

In [7]:
tf_lenta = tfv.fit_transform(lenta_news[:10])

In [8]:
tf_lenta

<10x2729 sparse matrix of type '<class 'numpy.float64'>'
	with 2773 stored elements in Compressed Sparse Row format>

In [13]:
list(tfv.vocabulary_.items())[:10]

[('испытанный украина', 791),
 ('украина первый', 2467),
 ('первый собственный', 1492),
 ('собственный крылатый', 2194),
 ('крылатый ракета', 918),
 ('ракета создать', 1904),
 ('создать кб', 2216),
 ('кб луч', 826),
 ('луч в', 965),
 ('в рамка', 216)]

In [14]:
# for news in tf_lenta:
#     print(sorted(news[0][:].todense()[0][0])[-5:])
    
    
freqwords = []
for i in tqdm(range(10)):

    tfs = [(word, tf_lenta[i, index]) for word, index in tfv.vocabulary_.items()
         if tf_lenta[i, index] != 0]
    fw = [w for w, f in sorted(tfs, key = lambda x: x[1], reverse = True)[:5]]
    freqwords.append(fw)


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 29.65it/s]


In [15]:
freqwords

[['ракета м',
  'собственный крылатый',
  'крылатый ракета',
  'противокорабельный ракета',
  'м комплекс'],
 ['на корточки',
  'русский гопник',
  'русский мат',
  'пользователь выразить',
  'выразить симпатия'],
 ['в результат',
  'сильный удар',
  'в американский',
  'американский штат',
  'штат вирджиния'],
 ['в франция',
  'евро рубль',
  'франция открыться',
  'открыться первый',
  'первый бордель'],
 ['процент россиянин',
  'данный показатель',
  'процент избиратель',
  'на выборы',
  'выборы президент'],
 ['сотрудник фсб',
  'фсб задержать',
  'задержать звезда',
  'звезда телесериал',
  'телесериал детектив'],
 ['на ближний',
  'ближний восток',
  'в регион',
  'совместный предприятие',
  'клиентский сервис'],
 ['причина исчезновение',
  'исчезновение рыболовецкий',
  'рыболовецкий судный',
  'судный восток',
  'восток с'],
 ['в сексуальный',
  'сексуальный домогательство',
  'в сексуальный домогательство',
  'в сша',
  'с оральный'],
 ['американский зоозащитный',
  'зоозащитн

Из теории вероятности используется несколько формул для расчета степени неслучайности для словосочетий.

$$MI(x, y)=log(\frac{f(x,y)*N}{f(x)*f(y)}),$$ 
где $f(x)$ и $f(y)$ - частоты встречаемости слов $x$ и $y$, а $f(x,y)$ - частота встречаемости фразы $x y$.

Для данной метрики берутся слова с самым большими значениями. Однако, так как нас интересует скорее ранжирование слов, чем абсолютные значения метрики, на практике удобнее использовать редуцированную формулу.

$$PMI(x, y)=\frac{f(x,y)}{f(x)*f(y)}$$

Еще одна формула:

$$t-score(x, y)=\frac{f(x,y)-\frac{f(x)*f(y)}{N}}{\sqrt{f(x,y)}},$$
где N - количество слов в коллекции.

О различиях между результатами можно прочитать [здесь](https://cyberleninka.ru/article/n/ot-kollokatsiy-k-konstruktsiyam/viewer) (можно начинать со второго раздела) или [здесь](https://www.researchgate.net/publication/340371729_K_voprosu_o_shodstve_mer_associacii_primenitelno_k_zadace_avtomaticeskogo_izvlecenia_glagolnyh_kollokacij).

Еще можно посмотреть информацию о [c-value](https://www.researchgate.net/publication/220387502_Automatic_Recognition_of_Multi-word_Terms_The_C-value_NC-value_Method).

Давайте посчитаем значения этих 

In [16]:
cntv1 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                        ngram_range=(1, 1))
cntv2 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                        ngram_range=(2, 2))

In [17]:
cnt_lenta_1 = cntv1.fit_transform(['\n'.join(lenta_news)])
cnt_lenta_2 = cntv2.fit_transform(['\n'.join(lenta_news)])

In [18]:
def calc_MI_tscore(cnt_lenta_1, cnt_lenta_2, cntv1, cntv2, thr):
    mis = {}
    t_scores = {}
    corp_len = cnt_lenta_1.sum()
    for pair, index2 in tqdm(cntv2.vocabulary_.items()):
        freq12 = cnt_lenta_2[0, index2]
        if freq12 < thr:
            continue
        words = pair.split(' ')
        freq1 = cnt_lenta_1[0, cntv1.vocabulary_[words[0]]]
        freq2 = cnt_lenta_1[0, cntv1.vocabulary_[words[1]]]
        mis[pair] = freq12 / (freq1 * freq2)
        t_scores[pair] = (freq12 - (freq1 * freq2) / corp_len) / np.sqrt(freq12)
    return mis, t_scores

In [19]:
mis, t_scores = calc_MI_tscore(cnt_lenta_1, cnt_lenta_2, cntv1, cntv2, 10)

100%|█████████████████████████████████| 139114/139114 [00:49<00:00, 2800.89it/s]


In [20]:
sorted(mis.items(), key=lambda x: x[1], reverse=True)[:100]

[('январярусский смертьзакать', 0.1),
 ('январясадизм какой-тобдсм-маска', 0.1),
 ('февралязолоть стволыонить', 0.1),
 ('упалокто ронять', 0.08333333333333333),
 ('демьян кудрявцев', 0.08333333333333333),
 ('возрождать сверхзвуковой', 0.07692307692307693),
 ('рен тв', 0.07692307692307693),
 ('сверхзвуковой ракетоносец', 0.06993006993006994),
 ('рекс тиллерсон', 0.06666666666666667),
 ('раюдин юсуфов', 0.06666666666666667),
 ('шахабас шахов', 0.06666666666666667),
 ('исаев раюдин', 0.0625),
 ('какой-тобдсм-маска суровый', 0.058823529411764705),
 ('февралякаждый своероссия', 0.058823529411764705),
 ('оливковый ветвь', 0.05803571428571429),
 ('суровый госпожа', 0.053475935828877004),
 ('алина загитов', 0.04807692307692308),
 ('шамиль исаев', 0.046875),
 ('саудовский аравия', 0.045454545454545456),
 ('хайат тахрир', 0.043478260869565216),
 ('настя рыбка', 0.04292929292929293),
 ('тахрир аш-ша', 0.03985507246376811),
 ('джабхата ан-нуср', 0.03764705882352941),
 ('переносный зенитно-ракетный

In [21]:
sorted(t_scores.items(), key=lambda x: x[1], reverse=True)[:100]

[('по тема', 29.034765192051626),
 ('материал по', 28.971646570039134),
 ('один из', 20.641825922221145),
 ('по слово', 19.904774681398106),
 ('в год', 18.689165929473734),
 ('о сообщать', 18.03640870166713),
 ('в время', 15.081959231549398),
 ('в пхенчхан', 13.18361587561156),
 ('по данные', 12.830756607324725),
 ('олимпийский игра', 12.623611308751203),
 ('на сайт', 12.49447966903917),
 ('игра в', 11.603784432349192),
 ('в тот', 11.46288796744148),
 ('российский спортсмен', 11.329909285207881),
 ('в частность', 11.327909201662566),
 ('в россия', 11.074044594859162),
 ('владимир путин', 10.893433483240813),
 ('тот число', 10.87885960684043),
 ('в результат', 10.863887968146972),
 ('ссылка на', 10.854642518158748),
 ('участие в', 10.849604563153544),
 ('с ссылка', 10.5992699826864),
 ('президент россия', 10.494159784392886),
 ('олимпийский комитет', 10.44377893676947),
 ('в конец', 10.435630586035296),
 ('риа новость', 10.382633704391456),
 ('тема декабрь', 10.081788632164717),
 ('кото

Несколько странные слова в топе в самом деле есть в тексте. Просто надо было при морфологическом анализе посмотреть, что это предсказанные слова, а не словарные.

Вообще, появление этих слов связано с тем, что MI поощряет появление редких слов. Например, если есть два слова, встретившихся вместе 1 раз, причем каждое слово встретилось только один раз (то есть они встретились только вместе), то MI = 1 / 1 * 1 = 1, то есть максимальному значению. Если есть два слова, встречающиеся только вместе, но 100 раз, MI = 100 / 100 * 100 = 0.01.

Посмотрим какие словосочетания встречаются больше 100 раз и какие у них получаются значения метрик.

In [22]:
mis_100, t_scores_100 = calc_MI_tscore(cnt_lenta_1, cnt_lenta_2, cntv1, cntv2, 100)

100%|█████████████████████████████████| 139114/139114 [00:49<00:00, 2838.95it/s]


In [23]:
sorted(mis_100.items(), key=lambda x: x[1], reverse=True)[:100]

[('риа новость', 0.004545071963639425),
 ('владимир путин', 0.0030199213297804847),
 ('уголовный дело', 0.0020567667626491155),
 ('олимпийский комитет', 0.00100116499199068),
 ('кроме тот', 0.0008683437619716511),
 ('олимпийский игра', 0.0008260985571496008),
 ('тот число', 0.000612948537862342),
 ('российский спортсмен', 0.00041872834439287586),
 ('тема декабрь', 0.00037104692316474483),
 ('один из', 0.00036986138681053936),
 ('с ссылка', 0.00026631036176885176),
 ('по тема', 0.00026117928293542726),
 ('материал по', 0.0002542138582869032),
 ('о сообщать', 0.00020736988915804155),
 ('по слово', 0.00017559151335825102),
 ('по данные', 0.0001726703722593569),
 ('ссылка на', 0.00016768222595450402),
 ('президент россия', 0.00016495267564615944),
 ('на сайт', 0.0001275990669222723),
 ('февраль сообщать', 0.00010398649205172555),
 ('в частность', 6.76795420041952e-05),
 ('на олимпиада', 6.220469672505794e-05),
 ('в пхенчхан', 6.033816388865958e-05),
 ('в ход', 5.669505962521295e-05),
 ('о 

In [24]:
sorted(t_scores_100.items(), key=lambda x: x[1], reverse=True)[:100]

[('по тема', 29.034765192051626),
 ('материал по', 28.971646570039134),
 ('один из', 20.641825922221145),
 ('по слово', 19.904774681398106),
 ('в год', 18.689165929473734),
 ('о сообщать', 18.03640870166713),
 ('в время', 15.081959231549398),
 ('в пхенчхан', 13.18361587561156),
 ('по данные', 12.830756607324725),
 ('олимпийский игра', 12.623611308751203),
 ('на сайт', 12.49447966903917),
 ('игра в', 11.603784432349192),
 ('в тот', 11.46288796744148),
 ('российский спортсмен', 11.329909285207881),
 ('в частность', 11.327909201662566),
 ('в россия', 11.074044594859162),
 ('владимир путин', 10.893433483240813),
 ('тот число', 10.87885960684043),
 ('в результат', 10.863887968146972),
 ('ссылка на', 10.854642518158748),
 ('участие в', 10.849604563153544),
 ('с ссылка', 10.5992699826864),
 ('президент россия', 10.494159784392886),
 ('олимпийский комитет', 10.44377893676947),
 ('в конец', 10.435630586035296),
 ('риа новость', 10.382633704391456),
 ('тема декабрь', 10.081788632164717),
 ('кото

Попробуем теперь избавиться от "странных" слов, получившихся из-за их склейки. PyMorphy2 помечает их, помещая данные о том, как проводился разбор в свойство `methods_stack`. Если слово было разобрано по словарю, будет использован `DictionaryAnalyzer`. Одно из многих значений для предсказанных слов - `FakeDictionary`. 

In [25]:
[type(x[0]) for x in morph.parse('русский')[0].methods_stack]

[pymorphy2.units.by_lookup.DictionaryAnalyzer]

In [26]:
[type(x[0]) for x in morph.parse('январярусский')[0].methods_stack]

[pymorphy2.units.by_analogy.KnownSuffixAnalyzer.FakeDictionary,
 pymorphy2.units.by_analogy.KnownSuffixAnalyzer]

In [27]:
pymorphy2.units.by_analogy.KnownSuffixAnalyzer.FakeDictionary in [type(x[0]) for x in morph.parse('январярусский')[0].methods_stack]

True

Перепишем функцию морфологического анализа так, чтобы она отсеивала несловарные слова (или хотя бы их часть).

In [28]:
imp_POS = ['ADJF', 'ADJS', 'NOUN', 'VERB', 'PRTF', 'PRTS', 'GRND', 'PREP']


def normalizePymorphy2(text):
    tokens = re.findall('[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        # Здесь мы проверяем, что слово было предсказано словарем.
        if str(pv[0].tag.POS) in imp_POS and \
          pymorphy2.units.by_lookup.DictionaryAnalyzer in [type(x[0]) for x in pv[0].methods_stack]:
            words.append(pv[0].normal_form)
            
    return words    


In [30]:
# lenta_news = [' '.join(normalizePymorphy2(news)) for news in tqdm(lenta_news.split(' '))]
# habr_news = [' '.join(normalizePymorphy2(news)) for news in tqdm(habr_news).split(' ')]

In [31]:
cntv12 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                        ngram_range=(1, 1))
cntv22 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                        ngram_range=(2, 2))

In [32]:
cnt_lenta_12 = cntv12.fit_transform(['\n'.join(lenta_news)])
cnt_lenta_22 = cntv22.fit_transform(['\n'.join(lenta_news)])

In [33]:
mis, t_scores = calc_MI_tscore(cnt_lenta_12, cnt_lenta_22, cntv12, cntv22, 10)

100%|█████████████████████████████████| 139114/139114 [00:50<00:00, 2767.27it/s]


In [34]:
sorted(mis.items(), key=lambda x: x[1], reverse=True)[:100]

[('январярусский смертьзакать', 0.1),
 ('январясадизм какой-тобдсм-маска', 0.1),
 ('февралязолоть стволыонить', 0.1),
 ('упалокто ронять', 0.08333333333333333),
 ('демьян кудрявцев', 0.08333333333333333),
 ('возрождать сверхзвуковой', 0.07692307692307693),
 ('рен тв', 0.07692307692307693),
 ('сверхзвуковой ракетоносец', 0.06993006993006994),
 ('рекс тиллерсон', 0.06666666666666667),
 ('раюдин юсуфов', 0.06666666666666667),
 ('шахабас шахов', 0.06666666666666667),
 ('исаев раюдин', 0.0625),
 ('какой-тобдсм-маска суровый', 0.058823529411764705),
 ('февралякаждый своероссия', 0.058823529411764705),
 ('оливковый ветвь', 0.05803571428571429),
 ('суровый госпожа', 0.053475935828877004),
 ('алина загитов', 0.04807692307692308),
 ('шамиль исаев', 0.046875),
 ('саудовский аравия', 0.045454545454545456),
 ('хайат тахрир', 0.043478260869565216),
 ('настя рыбка', 0.04292929292929293),
 ('тахрир аш-ша', 0.03985507246376811),
 ('джабхата ан-нуср', 0.03764705882352941),
 ('переносный зенитно-ракетный

In [35]:
sorted(t_scores.items(), key=lambda x: x[1], reverse=True)[:100]

[('по тема', 29.034765192051626),
 ('материал по', 28.971646570039134),
 ('один из', 20.641825922221145),
 ('по слово', 19.904774681398106),
 ('в год', 18.689165929473734),
 ('о сообщать', 18.03640870166713),
 ('в время', 15.081959231549398),
 ('в пхенчхан', 13.18361587561156),
 ('по данные', 12.830756607324725),
 ('олимпийский игра', 12.623611308751203),
 ('на сайт', 12.49447966903917),
 ('игра в', 11.603784432349192),
 ('в тот', 11.46288796744148),
 ('российский спортсмен', 11.329909285207881),
 ('в частность', 11.327909201662566),
 ('в россия', 11.074044594859162),
 ('владимир путин', 10.893433483240813),
 ('тот число', 10.87885960684043),
 ('в результат', 10.863887968146972),
 ('ссылка на', 10.854642518158748),
 ('участие в', 10.849604563153544),
 ('с ссылка', 10.5992699826864),
 ('президент россия', 10.494159784392886),
 ('олимпийский комитет', 10.44377893676947),
 ('в конец', 10.435630586035296),
 ('риа новость', 10.382633704391456),
 ('тема декабрь', 10.081788632164717),
 ('кото

Расмотрим теперь еще одну метрику - странность (weirdness). Для ее вычисления требуется рассчитать частоты сочетаний выбранной длинны в тематической коллекции и в еще одной - контрастивной, специально подобранном из другой предметной области. После этого рассчитывается отношение частот в тематическом и контрастивной коллекции. Для того, чтобы не делить на 0, применим сглаживание Лагранжа, то есть прибавим 1 к знаменателю.

Так как термины предметной области будут встречаться чаще в тематической коллекции, то самые большие значения будут у слов, похожих на термины. Верно и обратное, так как термины контрастивной коллекции чаще встречаются в ней, то близкие к 0 значения будут означать термины контрастивной коллеции.

In [36]:
hcntv1 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                         ngram_range=(1, 1))
hcntv2 = CountVectorizer(token_pattern="[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+-[а-яёА-ЯЁ]+|[а-яёА-ЯЁ]+", 
                         ngram_range=(2, 2))

In [37]:
cnt_habr_1 = hcntv1.fit_transform(['\n'.join(habr_news)])
cnt_habr_2 = hcntv2.fit_transform(['\n'.join(habr_news)])

In [38]:
weir_1 = {}
weir_2 = {}
for word, index in tqdm(hcntv1.vocabulary_.items()):
    if word in cntv1.vocabulary_.keys():
        w = cnt_habr_1[0, index] / (cnt_lenta_1[0, cntv1.vocabulary_[word]] + 1)
    else:
        w = cnt_habr_1[0, index]
    weir_1[word] = w
    
for word, index in tqdm(hcntv2.vocabulary_.items()):
    if word in cntv2.vocabulary_.keys():
        w = cnt_habr_2[0, index] / (cnt_lenta_2[0, cntv2.vocabulary_[word]] + 1)
    else:
        w = cnt_habr_2[0, index]
    weir_2[word] = w

100%|███████████████████████████████████| 73355/73355 [00:17<00:00, 4187.90it/s]


In [39]:
sorted(weir_1.items(), key=lambda x: x[1], reverse=True)[:100]

[('шрифт', 162),
 ('самохвалов', 153),
 ('граф', 130),
 ('настройка', 96),
 ('евтеев', 90),
 ('плагин', 82),
 ('браузер', 72),
 ('её', 68),
 ('метрика', 64),
 ('префикс', 57),
 ('фреймворк', 52),
 ('интерфейс', 50.5),
 ('оптимизация', 50),
 ('бурладянин', 50),
 ('шаблон', 47),
 ('скрипт', 46.5),
 ('стек', 45),
 ('хабра', 45),
 ('текстура', 45),
 ('трафаретный', 45),
 ('линейный', 44),
 ('буфер', 43),
 ('строка', 42.75),
 ('построение', 42),
 ('переменный', 38.0),
 ('тестировщик', 37),
 ('актор', 37),
 ('автоматизация', 35),
 ('буква', 34.5),
 ('вершина', 34.333333333333336),
 ('нейронный', 33),
 ('лямбда', 31),
 ('классификация', 30),
 ('преобразование', 30),
 ('иконка', 30),
 ('уза', 30),
 ('гб', 29),
 ('алгоритм', 28.0),
 ('паттерн', 28),
 ('датасет', 28),
 ('лог', 27),
 ('файл', 26.416666666666668),
 ('массив', 24.0),
 ('спецификация', 24),
 ('биткойна', 24),
 ('трекер', 23),
 ('антиква', 23),
 ('стейкхолдер', 23),
 ('репозиторий', 22.0),
 ('кэш', 22),
 ('лоадер', 22),
 ('фича', 21.

In [40]:
sorted(weir_2.items(), key=lambda x: x[1], reverse=True)[-100:]

[('первый ступень', 0.047619047619047616),
 ('расположить в', 0.047619047619047616),
 ('виноватый в', 0.047619047619047616),
 ('быть год', 0.047619047619047616),
 ('из сша', 0.047619047619047616),
 ('рубль в', 0.046511627906976744),
 ('в август', 0.045454545454545456),
 ('выступать на', 0.045454545454545456),
 ('в адрес', 0.045454545454545456),
 ('в май', 0.045454545454545456),
 ('вопрос о', 0.045454545454545456),
 ('в фильм', 0.045454545454545456),
 ('октябрь год', 0.044444444444444446),
 ('правило на', 0.043478260869565216),
 ('отказаться от', 0.043478260869565216),
 ('о писать', 0.0425531914893617),
 ('с по', 0.041666666666666664),
 ('инвестиция в', 0.041666666666666664),
 ('нарушение правило', 0.041666666666666664),
 ('ноябрь год', 0.041666666666666664),
 ('под руководство', 0.041666666666666664),
 ('в глава', 0.041666666666666664),
 ('тысяча человек', 0.04081632653061224),
 ('с январь', 0.04),
 ('по мнение', 0.04),
 ('доллар на', 0.04),
 ('период с', 0.04),
 ('о в', 0.039215686274

Если вам нужны термины длиннее, чем 2 слова,можно использовать [c-value](https://personalpages.manchester.ac.uk/staff/sophia.ananiadou/ijodl2000.pdf), которая определяет когда следует остановиться в цепочке слов.